In [ ]:
# threading 模块
import threading
import time


def loop():
    """  新的线程执行的代码 """
    n = 0
    while n < 5:
        print(n)
        nowThread = threading.current_thread()
        print(nowThread.name)  # loop_thread
        # 延迟1s执行
        time.sleep(1)
        n = n + 1


def useThread():
    """  使用线程来实现 """
    # 当前正在执行线程的名称
    nowThread = threading.current_thread()
    print(nowThread.name)  # MainThread 主线程
    # threading.Thread 创建一个线程 参数1：逻辑函数  参数2：线程名称
    t = threading.Thread(target=loop, name='loop_thread')
    # start启动线程
    t.start()
    # join 挂起线程
    t.join()


if __name__ == "__main__":
    useThread()

In [ ]:
import threading
import time


class LoopThread(threading.Thread):
    """自定义线程"""
    n = 0

    def run(self):
        while self.n < 5:
            print(self.n)
            nowThread = threading.current_thread()
            print(nowThread.name)
            time.sleep(1)
            self.n = self.n + 1


if __name__ == "__main__":
    t = LoopThread(name='loop_thread')
    t.start()
    t.join()


In [ ]:
# 多线程并发
import threading, time

# 假设 这是你的银行存款
balance = 0


def changeBalance(n):
    # 先存后取，结果应该为0
    global balance
    balance = balance + n
    time.sleep(2)
    balance = balance - n
    time.sleep(1)
    print('你当前成存款为：' + str(n) + ',余额为：' + str(balance))


class ChangeBalanceThread(threading.Thread):
    # 改变银行余额的线程
    def __init__(self, num, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num = num

    def run(self):
        for i in range(10):
            changeBalance(self.num)


if __name__ == "__main__":
    t1 = ChangeBalanceThread(5)
    t2 = ChangeBalanceThread(8)
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print(balance)

In [ ]:
# 线程中的锁
import threading, time

# 全局锁
myLock = threading.Lock()  # 资源不可重复锁定
youLock = threading.RLock()  # 资源可以多次锁定，但是也需要多次解锁才可以

# 假设 这是你的银行存款
balance = 0


def changeBalance(n):
    # 先存后取，结果应该为0
    global balance
    try:
        # 添加锁
        # myLock.acquire()
        # 资源已经被锁定了，Lock不能重复锁定，会造成死锁
        # # myLock.acquire()
        # RLock 可以对资源重复锁定，但是最终释放的时候也需要执行多次释放才可以
        youLock.acquire()
        youLock.acquire()
        balance = balance + n
        time.sleep(2)
        balance = balance - n
        time.sleep(1)
        print('你当前成存款为：' + str(n) + ',余额为：' + str(balance))
    finally:
        # 释放掉锁
        # myLock.release()
        youLock.release()
        youLock.release()


def changeBalance2(n):
    # 先存后取，结果应该为0
    global balance
    # 简化写法 with，这样就不需要每次添加锁并且最终去释放锁了
    with youLock:
        balance = balance + n
        time.sleep(2)
        balance = balance - n
        time.sleep(1)
        print('你当前成存款为：' + str(n) + ',余额为：' + str(balance))


class ChangeBalanceThread(threading.Thread):
    # 改变银行余额的线程
    def __init__(self, num, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num = num

    def run(self):
        for i in range(10):
            changeBalance(self.num)


if __name__ == "__main__":
    t1 = ChangeBalanceThread(5)
    t2 = ChangeBalanceThread(8)
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print(balance)


In [ ]:
# 线程池
import time
import threading
from concurrent.futures.thread import ThreadPoolExecutor
from multiprocessing.dummy import Pool


def run(n):
    """线程要做的事情"""
    # time.sleep(1)
    print(threading.current_thread().name, n)


def main():
    """使用传统的方法来做任务"""
    t1 = time.time()
    for n in range(100000):
        run(n)
    print(time.time() - t1)  # 0.8734416961669922


def useThreadMain():
    """使用线程优化任务"""
    # 资源有限最多跑10个线程
    t1 = time.time()
    ls = []
    for count in range(10):  # 10个线程
        for i in range(100000):
            t = threading.Thread(target=run, args=(i,))
            ls.append(t)
            t.start()

        for l in ls:
            l.join()
    print(time.time() - t1)


def usePoolMain():
    """使用线程池优化"""
    t1 = time.time()
    nList = range(100000)
    pool = Pool(10)
    pool.map(run, nList)
    pool.close()
    pool.join()
    print(time.time() - t1)


def useExectorMain():
    """使用ThreadPoolExector来优化"""
    t1 = time.time()
    nList = range(100000)
    with ThreadPoolExecutor(max_workers=10) as exector:
        exector.map(run, nList)
    print(time.time() - t1)


if __name__ == "__main__":
    # main()
    # useThreadMain()
    # usePoolMain()  # 1.7184045314788818 时间上多了一些，但是线程数量没有变的很多，复用了线程，数据量很多的时候就很有优势了
    useExectorMain()  # 3.512399911880493